In [2]:
# Partie 0
## Partie O.1 faite sur Powershell
## Partie 0.2 faite sur Powershell
# Partie 0.3 : Configuration de LiteLLM avec Claude API
# Utilisation de Claude Sonnet 4.5, le dernier modèle disponible

import os
from dotenv import load_dotenv
from litellm import completion

# Chargement du fichier .env
load_dotenv()

# Récupération de la clé API Claude
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

# Vérification que la clé est bien chargée
if anthropic_api_key:
    print("Clé Claude configurée avec succès.")
else:
    print("Erreur : clé Claude non trouvée dans .env")

# Test rapide de connexion avec Claude Sonnet 4.5
print("Test de connexion à Claude Sonnet 4.5...")

response = completion(
    model="claude-sonnet-4-5-20250929",
    messages=[{"role": "user", "content": "Réponds juste 'OK'"}],
    api_key=anthropic_api_key
)

print(f"Réponse : {response.choices[0].message.content}")
print("Claude Sonnet 4.5 est opérationnel !")

Clé Claude configurée avec succès.
Test de connexion à Claude Sonnet 4.5...
Réponse : OK
Claude Sonnet 4.5 est opérationnel !


In [4]:
# Partie 0.4 : Vérification complète du setup
# Contrôle que tous les composants sont opérationnels

import pandas as pd
import duckdb
import plotly.express as px

print("Vérification du setup complète :")
print("-" * 50)

# Vérifier que les modules importent correctement
libraries = ["pandas", "duckdb", "plotly", "litellm"]
for lib in libraries:
    try:
        __import__(lib)
        print(f"{lib}: OK")
    except ImportError:
        print(f"{lib}: ERREUR")

print("-" * 50)
print("Setup terminé ! Prêt pour l'exploration.\n")

Vérification du setup complète :
--------------------------------------------------
pandas: OK
duckdb: OK
plotly: OK
litellm: OK
--------------------------------------------------
Setup terminé ! Prêt pour l'exploration.



In [7]:
# Partie 1.2 : Chargement du dataset
# Chargement avec les bons paramètres de séparation

import pandas as pd

print("Chargement du dataset accidents corporels...")

# Chargement avec virgule comme séparateur et encodage latin-1
# on_bad_lines='skip' : ignore les lignes mal formatées s'il y en a
df = pd.read_csv('accidents.csv', encoding='latin-1', sep=',', on_bad_lines='skip')

print(f"Dataset chargé avec succès !")
print(f"Dimensions : {df.shape[0]} lignes, {df.shape[1]} colonnes")
print()

# Aperçu des premières lignes
print("Premières lignes du dataset :")
print(df.head())

print()
print("Noms des colonnes :")
print(df.columns.tolist())

Chargement du dataset accidents corporels...


C:\Users\hassa\AppData\Local\Temp\ipykernel_12888\545573002.py:10: DtypeWarning: Columns (15,29,31,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('accidents.csv', encoding='latin-1', sep=',', on_bad_lines='skip')


Dataset chargé avec succès !
Dimensions : 439332 lignes, 36 colonnes

Premières lignes du dataset :
   org  lum  agg  int  atm  col  com  dep  catr  infra  ...  numero  \
0    5    2    6    1  8.0  3.0   53   10   4.0    0.0  ...     NaN   
1    5    1    1    1  1.0  2.0   53   10   3.0    0.0  ...     NaN   
2    3    3    1    1  5.0  5.0  248   10   1.0    0.0  ...     NaN   
3    1    3    1    1  1.0  7.0  366   10   3.0    0.0  ...     NaN   
4    1    1    5    1  1.0  2.0    4   10   4.0    5.0  ...     NaN   

   distancemetre           libellevoie  coderivoli  grav  gps  lat  long  adr  \
0            0.0  23EME R.I. ( RUE DU)         NaN  0.43  NaN  NaN   NaN  NaN   
1            NaN                   NaN         NaN  0.43  NaN  NaN   NaN  NaN   
2            0.0                  A46N         NaN  0.43  NaN  NaN   NaN  NaN   
3            NaN                   NaN         NaN  0.43  NaN  0.0   0.0  NaN   
4            NaN                   NaN         NaN  0.43  NaN  0.0  

In [8]:
# Partie 2.1 : Création de la fonction d'assistance IA
# Fonction réutilisable pour poser des questions sur le dataset

from litellm import completion
import os

def ask_data_assistant(question: str, context: str = "") -> str:
    """
    Fonction pour poser une question à l'assistant IA sur les données
    
    Paramètres :
        question : la question à poser sur le dataset
        context : contexte du dataset (colonnes, statistiques, etc.)
    
    Retour :
        La réponse de l'assistant en tant que string
    """
    
    # Instruction système pour configurer le comportement de Claude
    system_prompt = """Tu es un expert en analyse de données.
    Aide à explorer et comprendre des datasets Open Data.
    Sois concis et pratique dans les réponses.
    Propose du code Python avec pandas ou plotly quand c'est pertinent."""
    
    # Construction du message pour Claude
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ]
    
    # Appel à Claude Sonnet 4.5
    response = completion(
        model="claude-sonnet-4-5-20250929",
        messages=messages,
        api_key=os.getenv('ANTHROPIC_API_KEY')
    )
    
    # Retour de la réponse
    return response.choices[0].message.content

print("Fonction d'assistance IA créée avec succès.")

Fonction d'assistance IA créée avec succès.


In [9]:
# Partie 2.2 : Génération du contexte du dataset
# Création d'un résumé structuré des données pour l'IA

def generate_dataset_context(df) -> str:
    """
    Fonction qui génère un résumé complet du dataset pour l'IA
    
    Paramètres :
        df : le DataFrame pandas contenant les données
    
    Retour :
        Un string avec toutes les informations du dataset
    """
    
    # Création du résumé structuré
    context = f"""
INFORMATIONS GÉNÉRALES :
- Nombre de lignes : {len(df)}
- Nombre de colonnes : {len(df.columns)}

COLONNES ET TYPES :
{df.dtypes.to_string()}

STATISTIQUES NUMÉRIQUES :
{df.describe().to_string()}

VALEURS MANQUANTES PAR COLONNE :
{df.isnull().sum().to_string()}

PREMIÈRES LIGNES :
{df.head(3).to_string()}
"""
    return context

# Génération du contexte
dataset_context = generate_dataset_context(df)

# Vérification
print("Contexte du dataset généré avec succès.")
print("Extrait (premières 500 caractères) :")
print(dataset_context[:500])

C:\Users\hassa\tp1-exploration\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)


Contexte du dataset généré avec succès.
Extrait (premières 500 caractères) :

INFORMATIONS GÉNÉRALES :
- Nombre de lignes : 439332
- Nombre de colonnes : 36

COLONNES ET TYPES :
org                int64
lum                int64
agg                int64
int                int64
atm              float64
col              float64
com                int64
dep                int64
catr             float64
infra            float64
voie             float64
v1               float64
v2                object
circ             float64
nbv              float64
pr                object


In [10]:
# Partie 2.3 : Exploration assistée par IA
# Posage des 5 questions principales sur le dataset

print("="*70)
print("QUESTION 1 : Description du dataset")
print("="*70)

q1 = ask_data_assistant(
    "Décris ce dataset en 3 phrases. Quel est son sujet principal ?",
    dataset_context
)
print(q1)

print("\n" + "="*70)
print("QUESTION 2 : Problèmes de qualité de données")
print("="*70)

q2 = ask_data_assistant(
    "Quels sont les 5 principaux problèmes de qualité de données que tu vois ?",
    dataset_context
)
print(q2)

print("\n" + "="*70)
print("QUESTION 3 : Proposer des analyses")
print("="*70)

q3 = ask_data_assistant(
    "Propose 3 analyses intéressantes qu'on pourrait faire avec ces données sur les accidents.",
    dataset_context
)
print(q3)

print("\n" + "="*70)
print("QUESTION 4 : Code de nettoyage")
print("="*70)

q4 = ask_data_assistant(
    "Génère le code Python pour nettoyer les valeurs manquantes de manière appropriée.",
    dataset_context
)
print(q4)

print("\n" + "="*70)
print("QUESTION 5 : Visualisation")
print("="*70)

q5 = ask_data_assistant(
    "Génère le code Plotly pour créer une visualisation pertinente des données.",
    dataset_context
)
print(q5)

QUESTION 1 : Description du dataset
# Dataset d'Accidents de la Route en France 🚗

Ce dataset contient **439 332 accidents de la route** recensés en France, avec 36 variables décrivant les circonstances de chaque accident (luminosité, conditions atmosphériques, type de route, localisation, etc.).

Les données couvrent la **gravité des accidents** (variable `grav`), le **nombre de victimes** (tués, blessés graves/légers via `ttue`, `tbg`, `tbl`), et les **caractéristiques géographiques et routières** (département, commune, catégorie de route, intersection, etc.).

Il s'agit manifestement de la **base BAAC (Bulletin d'Analyse des Accidents Corporels)** gérée par l'Observatoire National Interministériel de la Sécurité Routière, utilisée pour analyser la sécurité routière et identifier les facteurs d'accidents en France.

**📊 Points clés:**
- Près de 280k accidents n'ont pas de coordonnées GPS complètes
- Moyenne de 0.06 tué et 1.25 blessés par accident
- Données couvrant 97 départements d

In [11]:
# Partie 2.4 : Exercice libre - Questions supplémentaires
# Exploration créative du dataset avec 3 questions personnalisées

print("="*70)
print("QUESTION 6 : Question libre 1")
print("="*70)

q6 = ask_data_assistant(
    "Quels sont les départements (dep) avec le plus d'accidents et comment pourrait-on les identifier ?",
    dataset_context
)
print(q6)

print("\n" + "="*70)
print("QUESTION 7 : Question libre 2")
print("="*70)

q7 = ask_data_assistant(
    "Comment pourrait-on analyser la relation entre la gravité des accidents et les conditions météorologiques (colonne atm) ?",
    dataset_context
)
print(q7)

print("\n" + "="*70)
print("QUESTION 8 : Question libre 3")
print("="*70)

q8 = ask_data_assistant(
    "Quels insights intéressants pourraient aider à prévenir les accidents graves dans ce dataset ?",
    dataset_context
)
print(q8)

QUESTION 6 : Question libre 1
# Analyse des départements avec le plus d'accidents

Je vais t'aider à identifier les départements les plus accidentogènes. Voici comment procéder :

## 1. Code pour identifier les top départements

```python
import pandas as pd
import plotly.express as px

# Compter les accidents par département
accidents_par_dep = df['dep'].value_counts().reset_index()
accidents_par_dep.columns = ['departement', 'nb_accidents']

# Top 10 départements
top10 = accidents_par_dep.head(10)
print(top10)

# Visualisation
fig = px.bar(top10, 
             x='departement', 
             y='nb_accidents',
             title='Top 10 des départements avec le plus d\'accidents',
             labels={'departement': 'Département', 'nb_accidents': 'Nombre d\'accidents'})
fig.show()
```

## 2. Analyse enrichie avec taux d'accidents

```python
# Si tu as les populations départementales
populations = {
    # Exemple: '75': 2_165_000, '13': 2_044_000, etc.
}

# Calcul du taux pour 100k habi

In [12]:
# Partie 3.1 : Génération de la fiche technique
# Demande à Claude de générer une documentation complète du dataset

print("Génération de la fiche technique...")
print()

# Prompt pour générer la fiche technique au format Markdown
fiche_prompt = f"""
Génère une fiche technique complète pour ce dataset au format Markdown.

Structure attendue :
# Nom du dataset

## Informations générales
- Source : data.gouv.fr
- Licence : 
- Nombre de lignes : {len(df)}
- Nombre de colonnes : {len(df.columns)}

## Description
[Description en 2-3 paragraphes du contenu]

## Structure des données
[Tableau avec colonnes, types et descriptions]

## Qualité des données
### Points forts
- ...

### Points d'attention
- ...

## Cas d'usage potentiels
1. ...
2. ...
3. ...

## Analyse recommandée
[Recommandations pour exploiter le dataset]

Voici le contexte du dataset :
{dataset_context}
"""

# Appel à Claude pour générer la fiche
fiche_technique = ask_data_assistant(fiche_prompt)

print(fiche_technique)

Génération de la fiche technique...

# Accidents corporels de la circulation routière en France

## Informations générales
- **Source** : data.gouv.fr
- **Licence** : Licence Ouverte (à vérifier sur le portail)
- **Nombre de lignes** : 439 332
- **Nombre de colonnes** : 36
- **Période couverte** : Plusieurs années (à déterminer selon les données `org`)

## Description

Ce dataset recense les **accidents corporels de la circulation routière** survenus en France métropolitaine et dans les départements d'outre-mer. Chaque ligne correspond à un accident ayant fait au moins une victime (décédé, blessé hospitalisé ou blessé léger). Les données sont collectées par les forces de l'ordre à partir des procès-verbaux d'accidents.

Le fichier contient des informations détaillées sur les **circonstances des accidents** (conditions météorologiques, luminosité, type de collision), la **localisation géographique** (département, commune, adresse, coordonnées GPS) et les **conséquences humaines** (nombr

In [13]:
# Partie 3.2 : Sauvegarde de la fiche technique
# Enregistrement de la documentation générée en fichier Markdown

# Sauvegarde de la fiche technique en fichier .md
with open('FICHE_TECHNIQUE.md', 'w', encoding='utf-8') as fichier:
    fichier.write(fiche_technique)

print("Fiche technique sauvegardée dans FICHE_TECHNIQUE.md !")
print()
print("Fichier créé avec succès dans le dossier du projet.")

Fiche technique sauvegardée dans FICHE_TECHNIQUE.md !

Fichier créé avec succès dans le dossier du projet.


In [15]:
# Partie 3.3 : Création du README.md
# Documentation pour expliquer comment reproduire le TP

readme_content = """# TP1 - Exploration augmentée d'un dataset Open Data

## Dataset choisi
Accidents corporels de la circulation routière (data.gouv.fr)
- Source : https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/
- Nombre de lignes : 439 332
- Nombre de colonnes : 36

## Description
Ce TP explore un dataset Open Data contenant les informations sur les accidents corporels survenus sur les routes françaises. L'exploration est assistée par Claude Sonnet 4.5 pour identifier les caractéristiques, problèmes de qualité et analyses potentielles.

## Installation
```bash
cd tp1-exploration
uv sync
```

## Configuration
1. Créer un fichier `.env` à la racine du projet
2. Ajouter la clé API Anthropic :
```env
ANTHROPIC_API_KEY=clé_ici
```

## Utilisation
```bash
uv run jupyter notebook
```
Ouvrez le fichier `exploration.ipynb`

## Fichiers du projet
- `exploration.ipynb` : Notebook principal avec l'exploration du dataset
- `FICHE_TECHNIQUE.md` : Documentation détaillée du dataset
- `accidents.csv` : Fichier de données (à télécharger depuis data.gouv.fr)

## Contenu du TP
- Partie 0 : Setup de l'environnement
- Partie 1 : Chargement du dataset
- Partie 2 : Exploration assistée par IA (8 questions)
- Partie 3 : Génération de la fiche technique

## Outils utilisés
- Python 3.14
- pandas : manipulation de données
- duckdb : requêtes SQL
- plotly : visualisations
- litellm : accès à Claude API
- Claude Sonnet 4.5 : analyse assistée par IA

## Auteur
Hassan HOUSSEIN HOUMED

## Licence
Data : CC-BY
Notebook : Libre d'utilisation
"""

# Sauvegarde du README
with open('README.md', 'w', encoding='utf-8') as fichier:
    fichier.write(readme_content)

print("README.md créé avec succès !")

README.md créé avec succès !
